In [1]:
import pandas as pd


In [3]:
df_customers = pd.read_csv("..\\src\\credit_score\\data\\raw\\customers.csv")
df_loans = pd.read_csv("..\\src\\credit_score\\data\\raw\\loans.csv")
df_bureau = pd.read_csv("..\\src\\credit_score\\data\\raw\\bureau_data.csv")

In [4]:
df_customers.shape, df_loans.shape, df_bureau.shape

((50000, 12), (50000, 15), (50000, 8))

In [6]:
df = pd.merge(df_customers, df_loans, on="cust_id")
df.shape

(50000, 26)

In [7]:
df = pd.merge(df, df_bureau)
df.shape

(50000, 33)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 33 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   cust_id                      50000 non-null  object 
 1   age                          50000 non-null  int64  
 2   gender                       50000 non-null  object 
 3   marital_status               50000 non-null  object 
 4   employment_status            50000 non-null  object 
 5   income                       50000 non-null  int64  
 6   number_of_dependants         50000 non-null  int64  
 7   residence_type               49938 non-null  object 
 8   years_at_current_address     50000 non-null  int64  
 9   city                         50000 non-null  object 
 10  state                        50000 non-null  object 
 11  zipcode                      50000 non-null  int64  
 12  loan_id                      50000 non-null  object 
 13  loan_purpose    

In [10]:
df["default"] = df["default"].astype(int)
df["default"].value_counts()

default
0    45703
1     4297
Name: count, dtype: int64

In [12]:
from sklearn.model_selection import train_test_split
X = df.drop("default", axis=1)
y = df["default"]

# Split 70% train, 30% rest
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [13]:
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=1/3, random_state=42, stratify=y_test)

In [15]:
X_train.shape, X_val.shape, X_test.shape

((35000, 32), (10000, 32), (5000, 32))

In [16]:
import joblib

joblib.dump(X_test, "..\\src\\credit_score\\data\\test\\X_test.pkl")
joblib.dump(y_test, "..\\src\\credit_score\\data\\test\\y_test.pkl")

['..\\src\\credit_score\\data\\test\\y_test.pkl']